In [1]:
%matplotlib inline

# Guardar y cargar el modelo - 05 Exportar el modelo

## Entrenamiento de la red

Vamos a hacer de manera rápida el entrenamiento de una red (creada desde cero) para el conjunto de datos `CIFAR-10`

### Dataset

Descargamos y creamos el dataset

In [1]:
from torchvision import datasets
from torchvision.transforms import ToTensor

# Descargamos y creamos el dataset
dataset_train = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

dataset_test = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

Files already downloaded and verified
Files already downloaded and verified


### Dataloader

Creamos un dataloader

In [2]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64

train_dataloader = DataLoader(dataset_train, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(dataset_test, batch_size=BATCH_SIZE)

### Red

Creamos la red neuronal

In [3]:
from torch import nn

# Creamos la red neuronal desde cero
class NeuralNetworkFromScratch(nn.Module):
    def __init__(self):
        super(NeuralNetworkFromScratch, self).__init__()   # Se inicializa el módulo nn.Module
        self.flatten = nn.Flatten()             # Se crea una primera capa que aplana la imagen de entrada
        self.linear_relu_stack = nn.Sequential( # Se crea una módulo de arquitectura secuencial:
            nn.Linear(3*32*32, 512),                # Se añade una primera capa lineal que está preparada 
                                                    # para que le entre un vector de 28*28 (784)
                                                    # y sacará un vector de 512
            nn.ReLU(),                              # Se añade una no linealidad
            nn.Linear(512, 512),                    # Se añade una segunda capa lineal que le entran 512 
                                                    # datos y saca 512 datos
            nn.ReLU(),                              # Se añade una no linealidad
            nn.Linear(512, 10)                      # Se añade una tercera capa lineal que le entran 512 
                                                    # datos y saca un array de tamaño 10 (el número
                                                    # de etiquetas)
        )
        #self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.flatten(x)                         # Se pasa la imagen por la capa de aplanado
        logits = self.linear_relu_stack(x)          # Se pasa el vector resultante por la red
        #probs = self.softmax(logits)
        return logits

model_scratch = NeuralNetworkFromScratch()

### Función de pérdida

In [4]:
loss_fn = nn.CrossEntropyLoss()

### Optimizador

In [5]:
import torch

LR = 1e-2

optimizer = torch.optim.SGD(model_scratch.parameters(), lr=LR)

### Ciclo de entrenamiento

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # X and y to device
        X, y = X.to(device), y.to(device)

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            # X and y to device
            X, y = X.to(device), y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

model_scratch.to(device)

Using cuda device


NeuralNetworkFromScratch(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [8]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model_scratch, loss_fn, optimizer)
    test_loop(test_dataloader, model_scratch, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309021  [    0/50000]
loss: 2.273057  [ 6400/50000]
loss: 2.177390  [12800/50000]
loss: 2.192432  [19200/50000]
loss: 2.063237  [25600/50000]
loss: 2.049211  [32000/50000]
loss: 2.136148  [38400/50000]
loss: 1.967081  [44800/50000]
Test Error: 
 Accuracy: 29.9%, Avg loss: 1.953621 

Epoch 2
-------------------------------
loss: 1.998021  [    0/50000]
loss: 1.979714  [ 6400/50000]
loss: 1.755367  [12800/50000]
loss: 1.983313  [19200/50000]
loss: 1.972812  [25600/50000]
loss: 1.939373  [32000/50000]
loss: 1.986053  [38400/50000]
loss: 1.846420  [44800/50000]
Test Error: 
 Accuracy: 34.3%, Avg loss: 1.849024 

Epoch 3
-------------------------------
loss: 1.834622  [    0/50000]
loss: 1.842938  [ 6400/50000]
loss: 1.618993  [12800/50000]
loss: 1.863506  [19200/50000]
loss: 1.914225  [25600/50000]
loss: 1.861549  [32000/50000]
loss: 1.897058  [38400/50000]
loss: 1.767818  [44800/50000]
Test Error: 
 Accuracy: 36.6%, Avg loss: 1.779849 

Epoc

## Exportar el modelo

Hasta ahora teníamos el problema de que para cargar un modelo, una de dos, si guardábamos los pesos, necesitábamos declarar la red e instanciarla, y en el caso de guardar el modelo, no hacía falta instanciarla, pero si declarar la red.

Además que cuando queríamos volver a usar el modelo entrenado necesitábamos Pytorch, pero si en donde vamos a realizar la inferencia no tenemos Pytorch, o incluso peor, no tenemos ni Python estos métodos no nos valen

Para ello Pytorch también tiene soporte de exportación [ONNX](https://onnx.ai/) nativo.

ONNX es un formato abierto creado para representar modelos de aprendizaje automático. ONNX define un conjunto común de operadores (los componentes básicos de los modelos de aprendizaje automático y aprendizaje profundo) y un formato de archivo común para permitir que los desarrolladores de IA usen modelos con una variedad de marcos, herramientas, tiempos de ejecución y compiladores.

Sin embargo, dada la naturaleza dinámica del gráfico de ejecución de PyTorch, el proceso de exportación debe atravesar el gráfico de ejecución para producir un modelo ONNX persistente. Por esta razón, se debe pasar una variable de prueba del tamaño apropiado a la rutina de exportación (en nuestro caso, crearemos un tensor aleatorio ficticio del tamaño correcto):

In [14]:
import torch.onnx as onnx

path = "data/modelo.onnx"

batch = 8
input_image = torch.rand((batch,3,32,32))

# Llevamos la red neuronal a la CPU
model_scratch.to('cpu')

onnx.export(model_scratch, input_image, 'model.onnx')

El uso de [ONNX](https://onnx.ai/) es algo que se escapa al objetivo de este curso, por lo que si se quiere aprender cómo importar el modelo se recomienda visitar el [tutorial de ONNX](https://github.com/onnx/tutorials).